In [6]:
import numpy as np
n = 5
alpha = 0.1
cal_scrores = np.array([0.1,0.2,0.3,0.4,0.5])
q_level = np.ceil((n+1)*(1-alpha))/n
q_level = min(q_level, 1.0)                
print(q_level)
qhat = np.quantile(cal_scrores,q_level, method = 'higher')
print(qhat)
val_smx  = np.array([0.2,0.3,0.1,0.2,0.2])
predictions = val_smx >= (1-qhat)
print(predictions)

1.0
0.5
[False False False False False]
